In [16]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from spacy.lang.ru.stop_words import STOP_WORDS

df_items = pd.read_csv('./data/processed/items.csv')
df_users = pd.read_csv('./data/processed/users.csv')

## Items

In [17]:
df_items.sample(1)

,item_id,content_type,title,genres,age_rating,keywords,views
9468,11296,film,Георгий Седов,"биография, приключения, исторические, советски...",12,"Георгий, Седов, 1975, СССР, выживание, диких, ...",1


In [18]:
def tokenize(line: str):
    return [word for word in line.split(', ')
            if not word.isnumeric() and word not in {'', 'nan'}]

keywords_vectorizer = TfidfVectorizer(
    tokenizer=tokenize, token_pattern=None,
    max_features=200, stop_words=list(STOP_WORDS))

X_keywords = keywords_vectorizer.fit_transform(
    df_items['keywords'].values.astype('U')).todense()

X_content_type = (df_items['content_type'] == 'film').\
    astype(int).values.reshape(-1, 1)

age_rating_encoder = OneHotEncoder(sparse_output=False)
X_age_rating = age_rating_encoder.fit_transform(
    df_items['age_rating'].values.astype('U').reshape(-1, 1))

genres_vectorizer = TfidfVectorizer(tokenizer=tokenize, token_pattern=None)
X_genres = genres_vectorizer.fit_transform(
    df_items['genres'].values.astype('U')).todense()

In [19]:
keywords_vectorizer.get_feature_names_out()

array(['2010-е', '4к', 'автокатастрофа', 'автомобильная авария',
       'автомобильная погоня', 'англия', 'антигерой', 'арест', 'армия',
       'аэропорт', 'бандиты', 'бар', 'бассейн', 'беременность', 'битва',
       'близких', 'блокбастер', 'боевые искусства', 'бой', 'больница',
       'бомба', 'борьба', 'брак', 'брызги крови', 'великобритания',
       'вертолёт', 'вечеринка', 'взрыв', 'взрывающаяся машина',
       'винтовка', 'воин', 'война', 'волшебство', 'воровство', 'враги',
       'вражда', 'врач', 'выбор', 'выживание', 'выстрел в голову',
       'выстрел в грудь', 'выстрел в спину', 'гангстеры', 'германия',
       'героизм', 'герой', 'герой боевика', 'героя',
       'главный герой - женщина', 'гнев', 'гонка со временем',
       'город нью-йорк', 'девочка', 'девушка-подросток', 'дела', 'деньги',
       'держать на прицеле', 'детей', 'дети', 'добро против зла',
       'добрые', 'дождь', 'допрос', 'дочери', 'драка на кулаках',
       'дробовик', 'друг', 'дружба', 'друзья', 'женитьб

In [20]:
pd.Series(
    data=np.squeeze(np.asarray(X_keywords.sum(axis=0))),
    index=keywords_vectorizer.get_feature_names_out()
    ).sort_values(ascending=False)[:10]

россия                     1830.921565
соединенные штаты          1069.759158
отношения                   784.548648
франция                     783.907446
сша                         703.818336
любовь                      367.976010
соединенное королевство     353.638942
дружба                      340.458067
ссср                        321.341575
борьба                      291.650085
dtype: float64

In [21]:
X_movies = np.hstack([X_content_type, X_genres, X_age_rating, X_keywords])
X_movies.shape

(11909, 300)

## Users

In [22]:
df_users.sample(1)

,user_id,age,income,sex,kids_flg
118733,290705,age_45_54,income_20_40,М,1


In [23]:
age_encoder = OneHotEncoder(sparse_output=False)
X_age = age_encoder.fit_transform(
    df_users['age'].values.astype('U').reshape(-1, 1))

income_encoder = OneHotEncoder(sparse_output=False)
X_income = income_encoder.fit_transform(
    df_users['income'].values.astype('U').reshape(-1, 1))

X_sex = (df_users['sex'] == 'М').astype(int).values.reshape(-1, 1)

In [24]:
X_users = np.hstack([X_age, X_income, X_sex,
    df_users['kids_flg'].to_numpy().reshape(-1, 1)])
X_users.shape

(448798, 14)

In [25]:
np.savez_compressed('./data/processed/encoded_data',
                    X_users=X_users.astype(np.float32),
                    X_movies=X_movies.astype(np.float32))